# US Water Quality Data Portal
https://www.waterqualitydata.us/portal/

In [1]:
#Imports
import pandas as pd

## Bokeh map components
from bokeh.models import ColumnDataSource,WMTSTileSource
from bokeh.plotting import figure, show
from bokeh.tile_providers import *
from bokeh.io import output_notebook

#Display Bokeh in notebook
output_notebook()

Loading BokehJS ...

## Get data
These are files generated `from USWaterData-Scrape.ipynb`

In [2]:
#Convert to dataframe
dfSites = pd.read_csv("../Data/WaterQualityDataUS/DurhamSites.csv")
dfResults = pd.read_csv("../Data/WaterQualityDataUS/DurhamResults.csv",low_memory=False)
dfSamples = pd.read_csv("../Data/WaterQualityDataUS/DurhamSamples.csv",low_memory=False)
#dfActivity = pd.read_csv("../Data/WaterQualityDataUS/DurhamActivity.csv")

In [3]:
#Display the first record
df = dfSites.dropna(how='all',axis=1)
df.columns

Index(['OrganizationIdentifier', 'OrganizationFormalName',
       'MonitoringLocationIdentifier', 'MonitoringLocationName',
       'MonitoringLocationTypeName', 'MonitoringLocationDescriptionText',
       'HUCEightDigitCode', 'DrainageAreaMeasure/MeasureValue',
       'DrainageAreaMeasure/MeasureUnitCode',
       'ContributingDrainageAreaMeasure/MeasureValue',
       'ContributingDrainageAreaMeasure/MeasureUnitCode', 'LatitudeMeasure',
       'LongitudeMeasure', 'SourceMapScaleNumeric',
       'HorizontalAccuracyMeasure/MeasureValue',
       'HorizontalAccuracyMeasure/MeasureUnitCode',
       'HorizontalCollectionMethodName',
       'HorizontalCoordinateReferenceSystemDatumName',
       'VerticalMeasure/MeasureValue', 'VerticalMeasure/MeasureUnitCode',
       'VerticalAccuracyMeasure/MeasureValue',
       'VerticalAccuracyMeasure/MeasureUnitCode',
       'VerticalCollectionMethodName',
       'VerticalCoordinateReferenceSystemDatumName', 'CountryCode',
       'StateCode', 'CountyCode',

In [ ]:
df.ActivityMediaName.unique()

## Mapping

In [4]:
#Function to convert WGS84 points to Web Mercator
def wgs84_to_web_mercator(df, lon="lon", lat="lat"):
    """Converts decimal longitude/latitude to Web Mercator format"""
    import numpy as np
    k = 6378137
    df["x"] = df[lon] * (k * np.pi/180.0)
    df["y"] = np.log(np.tan((90 + df[lat]) * np.pi/360.0)) * k
    return df

In [5]:
#Generate colors
from bokeh.palettes import Category20
import itertools

def color_gen():
    yield from itertools.cycle(Category20[20])
color = color_gen()

In [7]:
#Add web mercator coordinates
df = wgs84_to_web_mercator(df,lon='LongitudeMeasure',lat='LatitudeMeasure')

In [ ]:
#Exaxmine the different types of Monitoring Locations
df.MonitoringLocationIdentifier.unique()

In [10]:
#Set the Bokeh CDS
sources = {}
for type in df.MonitoringLocationTypeName.unique().tolist():
    sources[type] = ColumnDataSource(df[df.MonitoringLocationTypeName == type])
sources.keys()

dict_keys(['Facility: Outfall', 'Atmosphere', 'Lake', 'Stream', 'Facility Other', 'Stream: Ditch', 'Well', 'Lake, Reservoir, Impoundment', 'Well: Hyporheic-zone well', 'Facility: Diversion', 'Reservoir', 'River/Stream', 'Facility: Pavement', 'Facility: Water-use establishment'])

In [14]:
from bokeh.palettes import Category20
xMin = df.x.min();xMax = df.x.max()
yMin = df.y.min();yMax = df.y.max()
p = figure(x_range=(xMin, xMax), y_range=(yMin, yMax),
           x_axis_type="mercator", y_axis_type="mercator")
p.add_tile(STAMEN_TERRAIN)
for i,type in enumerate(sources.keys()):
    source = sources[type]
    p.circle(x='x',y='y',source=source,color=Category20[16][i])
show(p);